<h1>Шаг 1: Открыть и изучить файл

In [1]:
#importing all libraries
import pandas as pd

#for splitting the data and counting score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#importing models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
#for checking of adequacy
from sklearn.dummy import DummyClassifier

In [2]:
#df = pd.read_csv('/Users/DanilBee/Desktop/Yandex_projects/mlTariffs/users_behavior.csv')
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


Увидел ,что данные все в правильном формате , то есть числовые и переменную ,которую нужно предсказывать в формате 1/0 , а не True/False. Так что готовы решать задачу бинарной классификации

<h1>Шаг 2: Разбить на выборки</h1>

In [4]:
#splitting into 2 sets 60% for the training and 40% for the test and validation
train_df, test_df = train_test_split(df, test_size = 0.4)

In [5]:
#splitting these 40% by half for the test and validation
test_df, valid_df = train_test_split(test_df, test_size = 0.5)

In [6]:
print('Длина выборки тренировки :', train_df.shape[0])
print('Длина выборки теста :', test_df.shape[0])
print('Длина выборки валидации :', valid_df.shape[0])

Длина выборки тренировки : 1928
Длина выборки теста : 643
Длина выборки валидации : 643


In [7]:
#making features and target for each set

#train's features and target
train_features = train_df.drop('is_ultra', axis = 1)
train_target = train_df['is_ultra']

#test's features and target
test_features = test_df.drop('is_ultra', axis = 1)
test_target = test_df['is_ultra']

#validation's features and target
valid_features = valid_df.drop('is_ultra', axis = 1)
valid_target = valid_df['is_ultra']

Разбили на 3 выборки , то есть дали 60 % данных на тренировку модели , и остальные 40% поделили поровну между валидационной выборкой и тестовой. Также для каждой выборки поделили данные на: признаки и на целевой признак

<h1>Шаг 3: Исследовать данные на выборки</h1>

In [8]:
#Decision Tree model
best = 0
i = 0
for index in range(1,50,1):
    treeModel = DecisionTreeClassifier(max_depth = index ,random_state = 42)
    
    treeModel.fit(train_features, train_target)
    
    treePrediction = treeModel.predict(valid_features)
    
    acc = accuracy_score(valid_target,treePrediction)
    
    if acc > best:
        best = acc
        i = index
print('best accuracy for Decision Tree on validation set =',best,'with the {} max depth'.format(i))

best accuracy for Decision Tree on validation set = 0.7869362363919129 with the 6 max depth


In [10]:
best = 0
i = 0
d = 0
for estim in range (1, 50, 1):
    for depth in range(1, 10 ,1):
        forestModel = RandomForestClassifier(max_depth = depth,n_estimators = estim, random_state = 42)
    
        forestModel.fit(train_features, train_target)
    
        forestPrediction = forestModel.predict(valid_features)
    
        acc = accuracy_score(valid_target, forestPrediction)
    
        if acc > best:
            best = acc
            i = estim
            d = depth
print('best accuracy for Random Forest on validation set =',best,'with the {} estimators'.format(i),'and depth = ',d)

best accuracy for Random Forest on validation set = 0.8211508553654744 with the 12 estimators and depth =  8


Исследовали данные на выборках и исследовали только на 2 моделях , так как в них мы исследуем поведение модели от гиперпараметров ,которые мы меняем в цикле и ищем лучшие показатели. Также в случае со случайным лесом провели достаточно емкое обучение , но у нас нету задачи повысить скорость , а ,скорей, проделать более емкое исследование,поэтому проводил исследование по комбинации 2 гиперпараметров


In [17]:
index

49

<h1>Шаг 4: Проверить модели на тестовой выборки</h1>

In [12]:
#checking for the effectivness of parameters figured out on validation set for the RandomForestClassifier
forestModel = RandomForestClassifier(max_depth = 8, n_estimators = 49, random_state = 42)
forestModel.fit(train_features, train_target)
predFMtest = forestModel.predict(test_features)
print("Score of RandomForest on the testing data is:", accuracy_score(test_target, predFMtest))

Score of RandomForest on the testing data is: 0.8055987558320373


In [13]:
#checking for the effectivness of parameters figured out on validation set for the DecisionTreeClassifier
treeModel = DecisionTreeClassifier(max_depth = 3, random_state = 42)
treeModel.fit(train_features, train_target)
predTMtest = treeModel.predict(test_features)
print("Score of DecisionTree on the testing data is:", accuracy_score(test_target, predTMtest))

Score of DecisionTree on the testing data is: 0.7838258164852255


In [14]:
#we don't tweak any parameters in Logistic Regresson 
#so we actually can instead of tweaking - just make training and testing sets 
#and improve the accuracy that way so we have more data to train the model
trainLR_df, testLR_df = train_test_split(df, test_size = 0.3)

trainLR_features = trainLR_df.drop('is_ultra', axis = 1)
trainLR_target = trainLR_df['is_ultra']
testLR_features = testLR_df.drop('is_ultra', axis = 1)
testLR_target = testLR_df['is_ultra']

logisticModel = LogisticRegression(random_state = 42)

logisticModel.fit(trainLR_features, trainLR_target)

predLRtest = logisticModel.predict(testLR_features)

print("Score of LogisticRegression on the testing data is:", accuracy_score(testLR_target, predLRtest))

Score of LogisticRegression on the testing data is: 0.6911917098445596


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Модель после исследования не дала переобучаемость , ведь сохранила достаточно близку точность на тестовых данных к валидационным. В случае с логистической регрессией , для нее провели отдельный тест , так как она не нужная в валидационных данных , ведь там нету гиперпараметров , поэтому можно улучшить ее точность за счет увелечения объема данных , не используя валидационный сет. Таким образом получили 3 достаточно точных модели , где фаворитом в качестве выступает Случайный лес 79%, но с наименьшей скоростью. Не сильно ему уступает также Дерево решений 78% и логистическая регрессия еле как набрала необходимый результат 75%

<h1>Шаг 5: Проверить модели на адекватность</h1>

In [15]:
#training the dummy model to compare
dummyModel = DummyClassifier(strategy = 'most_frequent',random_state = 42)
dummyModel.fit(train_features, train_target)
predDummytest = dummyModel.predict(test_features)
print('The most frequent correct answer model will have accuracy of', accuracy_score(test_target,predDummytest))

The most frequent correct answer model will have accuracy of 0.6936236391912908


Так как логистическую регрессию тренировали и проверяли с другим объемом данных - поменяем на такие же объемы данных как и в ней для модели проверки

In [16]:
dummyModel2 = DummyClassifier(strategy = 'most_frequent',random_state = 42)
dummyModel2.fit(trainLR_features, trainLR_target)
predDummytest2 = dummyModel2.predict(testLR_features)
print('The most frequent correct answer model will have accuracy of', accuracy_score(testLR_target,predDummytest2))

The most frequent correct answer model will have accuracy of 0.6860103626943005


Из первой простой модели для сравнения Дерева решений и Случайного леса получили ,что обе модели адекватные 78% и 79%  против 68% у простой модели. Так как для логистической использовали другой подход - создали вторую просту модель на основе объема логистической регрессии и также получили ,что логистическая модель прошла проверку на адекватность 75% против 69%